# Assignment_1
## Feature sign search algorithm

##### A:2D Matrix of basis vectors
##### y:input vector to be sparsely encoded
##### gamma: sparsity co-efficient
##### x: output sparse encoding of y on the basis A with sparsity co-efficient gamma


In [1]:
import numpy as np

def feature_sign_search(A, y, gamma):
    
    effective_zero = 1e-18
    
    A_t__A = np.dot(A.T, A)

    A_t__y = np.dot(A.T, y)
   
    x = np.zeros(A_t__A.shape[0])
    theta = np.zeros(A_t__A.shape[0], dtype=np.int8)
    active_set = set()
    z_opt = np.inf
    nz_opt = 0
    gradient = - 2 * A_t__y  # + 2 * np.dot(A_t__A, x)
    max_gradient_zero = np.argmax(np.abs(gradient))
    
    sds = np.dot(y.T, y)
    while z_opt > gamma or not np.allclose(nz_opt, 0):
        if np.allclose(nz_opt, 0):
            candidate = np.argmax(np.abs(gradient) * (theta == 0))
            if gradient[candidate] > gamma:
                theta[candidate] = -1.
                x[candidate] = 0.
                active_set.add(candidate)
            elif gradient[candidate] < -gamma:
                theta[candidate] = 1.
                x[candidate] = 0.
                active_set.add(candidate)
            if len(active_set) == 0:
                break
        indices = np.array(sorted(active_set))
        A_t__A_new = A_t__A[np.ix_(indices, indices)]
        A_t__y_new = A_t__y[indices]
        theta_ = theta[indices]
        rhs = A_t__y_new - gamma * theta_ / 2
        x_new = np.linalg.solve(np.atleast_2d(A_t__A_new), rhs)
        new_theta = np.sign(x_new)
        _oldsol = x[indices]
        sign_flips = np.where(abs(new_theta - theta_) > 1)[0]
        print(new_theta,theta_,indices,x_new)
        if len(sign_flips) > 0:
            best_obj = np.inf
            best_curr = None
            best_curr = x_new
            best_obj = (sds + (np.dot(x_new,
                                      np.dot(A_t__A_new, x_new))
                        - 2 * np.dot(x_new, A_t__y_new))
                        + gamma * abs(x_new).sum())
            for idx in sign_flips:
                a = x_new[idx]
                b = _oldsol[idx]
                prop = b / (b - a)
                curr = _oldsol - prop * (_oldsol - x_new)
                cost = sds + (np.dot(curr, np.dot(A_t__A_new, curr))
                              - 2 * np.dot(curr, A_t__y_new)
                              + gamma * abs(curr).sum())
                if cost < best_obj:
                    best_obj = cost
                    best_prop = prop
                    best_curr = curr
        else:
            best_curr = x_new
        x[indices] = best_curr
        zeros = indices[np.abs(x[indices]) < effective_zero]
        x[zeros] = 0.
        theta[indices] = np.int8(np.sign(x[indices]))
        
        active_set.difference_update(zeros)
        gradient = - 2 * A_t__y + 2 * np.dot(A_t__A, x)
        print(gradient[theta == 0])
        if len(gradient[theta == 0]) == 0:
             break
        z_opt = np.max(abs(gradient[theta == 0]))
        nz_opt = np.max(abs(gradient[theta != 0] + gamma * theta[theta != 0]))
    return x

#### Example

In [2]:
input = np.array([1,2,3,4,5])
basis = np.array([[0,0,0,0,5],[1,0,0,0,0],[0,1,0,0,0],[0,1.5,0,0,0],[0,0,3,4,0],[0,0,0,1,0],[0,0,0,0,1]]).T
sparsity = 0.01
sparse_encoding = feature_sign_search(basis,input,sparsity)
print(u'Original input vector\n')
print(input)
print(u'Reconstructed vector\n')
print(np.matmul(basis,sparse_encoding))
print(u'Sparse representation\n')
print(sparse_encoding)

[ 1.] [1] [0] [ 0.9998]
[ -2.00000000e+00  -4.00000000e+00  -6.00000000e+00  -5.00000000e+01
  -8.00000000e+00  -2.00000000e-03]
[ 1.  1.] [1 1] [0 4] [ 0.9998  0.9998]
[ -2.00000000e+00  -4.00000000e+00  -6.00000000e+00  -1.60000000e-03
  -2.00000000e-03]
[ 1.  1.  1.] [1 1 1] [0 3 4] [ 0.9998      1.33111111  0.9998    ]
[ -2.00000000e+00  -6.66666667e-03  -1.60000000e-03  -2.00000000e-03]
[ 1.  1.  1.  1.] [1 1 1 1] [0 1 3 4] [ 0.9998      0.995       1.33111111  0.9998    ]
[-0.00666667 -0.0016     -0.002     ]
Original input vector

[1 2 3 4 5]
Reconstructed vector

[ 0.995       1.99666667  2.9994      3.9992      4.999     ]
Sparse representation

[ 0.9998      0.995       0.          1.33111111  0.9998      0.          0.        ]
